In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import math
import json
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns
# import missingno as msnco

In [2]:
import warnings

# 경고 메시지를 무시하고 숨기거나
warnings.filterwarnings(action='ignore')

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_df = pd.read_csv('sample_submission.csv')

In [8]:
len(train_df['Origin_Airport'].unique())

374

In [9]:
len(train_df['Destination_Airport'].unique())

375

In [4]:
test_df.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    108984
Estimated_Arrival_Time      109048
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                106505
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           106523
Distance                         0
Airline                     106527
Carrier_Code(IATA)          108993
Carrier_ID(DOT)             109006
Tail_Number                      0
dtype: int64

In [5]:
train_df

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN


In [6]:
state_list = train_df['Origin_State'].unique().tolist()

In [7]:
#출발 a. 도착 a일수가이ㅏㅆ을까 --> 같을 수 있음
train_df[train_df['Origin_State']==train_df['Destination_State']]

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN
35,TRAIN_000035,12,22,630.0,810.0,0,0,SFO,14771,California,SNA,14908,California,372.0,Horizon Air,AS,19687.0,N650QX,NaN
43,TRAIN_000043,9,30,1325.0,1506.0,0,0,SAN,14679,California,SFO,14771,California,447.0,United Air Lines Inc.,UA,NaN,N471UA,Not_Delayed
63,TRAIN_000063,12,28,1820.0,1925.0,0,0,OAK,13796,California,BUR,10800,California,325.0,Southwest Airlines Co.,WN,19393.0,N777QC,Not_Delayed
64,TRAIN_000064,9,15,1918.0,2040.0,0,0,OAJ,13795,North Carolina,CLT,11057,North Carolina,191.0,Comair Inc.,AA,20397.0,N244PS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999935,TRAIN_999935,7,7,650.0,755.0,0,0,SJC,14831,California,BUR,10800,California,296.0,Southwest Airlines Co.,WN,19393.0,N7865A,NaN
999940,TRAIN_999940,4,17,717.0,804.0,0,0,KOA,12758,Hawaii,HNL,12173,Hawaii,163.0,Hawaiian Airlines Inc.,HA,19690.0,N477HA,Not_Delayed
999942,TRAIN_999942,7,15,1950.0,2220.0,0,0,ELP,11540,Texas,SAT,14683,Texas,496.0,Southwest Airlines Co.,WN,19393.0,N7741C,NaN
999969,TRAIN_999969,10,10,940.0,1056.0,0,0,MFE,13256,Texas,IAH,12266,Texas,316.0,Mesa Airlines Inc.,NaN,20378.0,N89321,Delayed


In [8]:
# 그렇다면 for o in state_list / for d in state_list  라고 하면 
# 출발주 o 도착주d 인  경우를 출력해서 
# 

In [ ]:
# time_delay를 통해 time_diff를 계산하기 위해  먼저 time_diff 컬럼 생성 
df['time_diff'] = np.nan


# 구한 시차를 df에 time_diff 컬럼에 넣어주기 
for i in range(len(df)):   
    if (df['Origin_State'].iloc[i],df['Destination_State'].iloc[i]) in time_delay.keys() :
        df['time_diff'].iloc[i] = time_delay[df['Origin_State'].iloc[i],df['Destination_State'].iloc[i]]
    else : 
        df['time_diff'].iloc[i] == 'time'

In [ ]:
for i in state_list:
    for j in state_list:
        if df[train_df['Origin_State']==i] and 

In [ ]:
#도착시간이 출발시간보다 커야함. 출발A 도착B. 
if train_df['Origin_State']==a and train_df['Destination_State']==b : #a와 b는 state_list 안의 원소
    
    

In [ ]:
if train_df['Origin_State']=='California' and train_df['Destination_State']=='California': #a와 b는 state_list 안의 원소
    train

In [ ]:
def find_time_lag(train_df, state):
    OA = train_df[train_df['Origin_State'] == state]
    predict_st = OA.Destination_State.unique().tolist()
    DA = train_df[train_df['Destination_State'] == state]
    time_delay_dict = {}
    for st in predict_st:
        st_oa = OA[OA['Destination_State'] == st]
        st_oa_diff = st_oa.Estimated_Arrival_Time.mean() - st_oa.Estimated_Departure_Time.mean()
        st_da = DA[DA['Origin_State'] == st]
        st_da_diff = st_da.Estimated_Arrival_Time.mean() - st_da.Estimated_Departure_Time.mean()
        time_delay = st_da_diff - st_oa_diff
        if not np.isnan(time_delay):
            time_delay_dict[state, st] = -round(time_delay/2/60)
    
    return time_delay_dict

time_delay = dict()
for i in train_df['Origin_State'].unique().tolist():
    time_delay.update(find_time_lag(train_df, i))
    
print(time_delay)

In [ ]:
def time_diff(train_df, state):
    OA = train_df[train_df['Origin_State'] == state]
    destination_list = OA.Destination_State.unique().tolist()
    time_diff_dict = {}

    for d in destination_list:
        st_oa = OA[OA['Destination_State']==d]
        time_diff_dict[state,d] = 

In [35]:
OA = train_df[train_df['Origin_State'] == 'California']
destination_list = OA.Destination_State.unique().tolist()
time_diff_dict = {}
distance_dict = {}
for i in destination_list:
    st_oa = OA[OA['Destination_State']==i]
    time_diff_dict['California',i] = (st_oa['Estimated_Arrival_Time']-st_oa['Estimated_Departure_Time']).mean()
    distance_dict['California',i] = st_oa['Distance'].mean()

In [22]:
#출발 state, 도착 d일 때 두 시간의 차이를 time_diff_dict에 넣고, 두 state의 거리의 평균을 distance_mean_dict에 넣기 
def time_diff(train_df, state):
    OA = train_df[train_df['Origin_State']==state]
    destination_list = OA.Destination_State.unique().tolist()
    time_diff_dict = {}
    distance_mean_dict = {}
    for d in destination_list:
        st_oa = OA[OA['Destination_State']==d]
        time_diff_dict[state,d] = (st_oa['Estimated_Arrival_Time'] - st_oa['Estimated_Departure_Time']).mean
        distance_mean_dict[state,d] = st_oa['Distance'].mean()
    
    return time_diff_dict, distance_mean_dict

In [ ]:
time_diff ={}
distance_mean ={}
for i in state_list:
    time_diff.update(find_time_lag(train_df, i))
    distance_mean.update(fi)

In [32]:
time_diff_dict = {}
distance_mean_dict = {}
def time_diff(train_df, state):
    OA = train_df[train_df['Origin_State']==state]
    destination_list = OA.Destination_State.unique().tolist()
#     time_diff_dict = {}
#     distance_mean_dict = {}
    for d in destination_list:
        st_oa = OA[OA['Destination_State']==d]
        time_diff_dict[state,d] = (st_oa['Estimated_Arrival_Time'] - st_oa['Estimated_Departure_Time']).mean
        distance_mean_dict[state,d] = st_oa['Distance'].mean()
    
    return time_diff_dict #distance_mean_dict

In [28]:
time_diff_dict = {}
distance_mean_dict = {}
for i in state_list :
    time_diff(train_df, i)

In [33]:
time_diff_dict = {}
distance_mean_dict = {}

In [ ]:
#출발 a, 도착 b 

In [55]:
time_diff_dict = {}
distance_mean_dict = {}
def time_diff(train_df, state):
    OA = train_df[train_df['Origin_State']==state]
    destination_list = OA.Destination_State.unique().tolist()
#     time_diff_dict = {}
#     distance_mean_dict = {}
    for d in destination_list:
        st_oa = OA[OA['Destination_State']==d]
        time_diff = (st_oa['Estimated_Arrival_Time'] - st_oa['Estimated_Departure_Time']).mean()
        time_diff_dict[state,d] = (time_diff)
        distance_mean_dict[state,d] = st_oa['Distance'].mean()
    
    return time_diff_dict, distance_mean_dict

In [53]:
df = train_df[train_df['Estimated_Arrival_Time']>train_df['Estimated_Departure_Time']]
df

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN
6,TRAIN_000006,1,20,1742.0,1903.0,0,0,EWR,11618,New Jersey,BOS,10721,Massachusetts,200.0,United Air Lines Inc.,UA,NaN,N66825,Not_Delayed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN


In [56]:
time_diff(df, 'California')

{('California', 'New Jersey'): 834.9496788008565,
 ('California', 'California'): 135.90967370756965,
 ('California', 'Utah'): 282.1205533596838,
 ('California', nan): nan,
 ('California', 'Missouri'): 550.367231638418,
 ('California', 'Oregon'): 196.42008395221183,
 ('California', 'Arizona'): 190.88584474885846,
 ('California', 'Indiana'): 717.828125,
 ('California', 'New Mexico'): 302.20302375809933,
 ('California', 'Hawaii'): 310.53544620517096,
 ('California', 'Texas'): 515.201308411215,
 ('California', 'Nevada'): 129.29365079365078,
 ('California', 'Florida'): 803.2478314745973,
 ('California', 'Illinois'): 610.0054995417048,
 ('California', 'Tennessee'): 607.9586563307494,
 ('California', 'Washington'): 245.6105748135147,
 ('California', 'Kentucky'): 729.4772727272727,
 ('California', 'Minnesota'): 560.8126984126984,
 ('California', 'Colorado'): 339.4112295295035,
 ('California', 'Massachusetts'): 856.545038167939,
 ('California', 'New York'): 849.4690415606447,
 ('California', 'G

In [57]:
state_list = train_df['Origin_State'].unique().tolist()

In [58]:
for i in state_list:
    time_diff(df, i)

In [59]:
time_diff_dict

{('California', 'New Jersey'): 834.9496788008565,
 ('California', 'California'): 135.90967370756965,
 ('California', 'Utah'): 282.1205533596838,
 ('California', nan): nan,
 ('California', 'Missouri'): 550.367231638418,
 ('California', 'Oregon'): 196.42008395221183,
 ('California', 'Arizona'): 190.88584474885846,
 ('California', 'Indiana'): 717.828125,
 ('California', 'New Mexico'): 302.20302375809933,
 ('California', 'Hawaii'): 310.53544620517096,
 ('California', 'Texas'): 515.201308411215,
 ('California', 'Nevada'): 129.29365079365078,
 ('California', 'Florida'): 803.2478314745973,
 ('California', 'Illinois'): 610.0054995417048,
 ('California', 'Tennessee'): 607.9586563307494,
 ('California', 'Washington'): 245.6105748135147,
 ('California', 'Kentucky'): 729.4772727272727,
 ('California', 'Minnesota'): 560.8126984126984,
 ('California', 'Colorado'): 339.4112295295035,
 ('California', 'Massachusetts'): 856.545038167939,
 ('California', 'New York'): 849.4690415606447,
 ('California', 'G

## 출발A 도착B  state간의 시간 차이와 거리 평균 딕셔너리에 저장

In [60]:
#도착시간이 출발시간보다 큰 경우만
df = train_df[train_df['Estimated_Arrival_Time']>train_df['Estimated_Departure_Time']]
df

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN
6,TRAIN_000006,1,20,1742.0,1903.0,0,0,EWR,11618,New Jersey,BOS,10721,Massachusetts,200.0,United Air Lines Inc.,UA,NaN,N66825,Not_Delayed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN


In [95]:
#시간차이와 거리평균 딕셔너리 형태로 저장
time_diff_dict = {}
distance_mean_dict = {}
def time_diff(train_df, state):
    OA = train_df[train_df['Origin_State']==state]
    destination_list = OA.Destination_State.unique().tolist()
#     time_diff_dict = {}
#     distance_mean_dict = {}
    for d in destination_list:
        st_oa = OA[OA['Destination_State']==d]
        time_diff = (st_oa['Estimated_Arrival_Time'] - st_oa['Estimated_Departure_Time']).mean()
        time_diff_dict[state,d] = (time_diff)
        distance_mean_dict[state,d] = (st_oa['Distance'].mean())
    
    return time_diff_dict, distance_mean_dict

In [96]:
for i in state_list:
    time_diff(df, i)

In [97]:
time_diff_dict

{('Oklahoma', 'Texas'): 130.94105409153954,
 ('Oklahoma', 'Colorado'): 75.51356589147287,
 ('Oklahoma', nan): nan,
 ('Oklahoma', 'Georgia'): 296.15021459227466,
 ('Oklahoma', 'Illinois'): 206.05176470588236,
 ('Oklahoma', 'New Jersey'): 418.61538461538464,
 ('Oklahoma', 'California'): 148.82222222222222,
 ('Oklahoma', 'Virginia'): 370.11267605633805,
 ('Oklahoma', 'Florida'): 324.61224489795916,
 ('Oklahoma', 'Missouri'): 129.19354838709677,
 ('Oklahoma', 'North Carolina'): 357.2282608695652,
 ('Oklahoma', 'Minnesota'): 213.625,
 ('Oklahoma', 'Arizona'): 83.3558282208589,
 ('Oklahoma', 'Utah'): 173.31460674157304,
 ('Oklahoma', 'Nevada'): 74.14685314685315,
 ('Oklahoma', 'Maryland'): 372.1333333333333,
 ('Oklahoma', 'Michigan'): 353.1081081081081,
 ('Oklahoma', 'Pennsylvania'): 419.71875,
 ('Oklahoma', 'Washington'): 221.1851851851852,
 ('Oklahoma', 'Tennessee'): 140.0,
 ('Illinois', 'Utah'): 261.23646723646726,
 ('Illinois', 'Missouri'): 141.63670647391578,
 ('Illinois', 'Kentucky'): 

In [98]:
distance_mean_dict

{('Oklahoma', 'Texas'): 311.22884882108184,
 ('Oklahoma', 'Colorado'): 514.8798449612403,
 ('Oklahoma', nan): nan,
 ('Oklahoma', 'Georgia'): 715.0729613733905,
 ('Oklahoma', 'Illinois'): 645.5929411764706,
 ('Oklahoma', 'New Jersey'): 1325.0,
 ('Oklahoma', 'California'): 1274.2518518518518,
 ('Oklahoma', 'Virginia'): 1132.0845070422536,
 ('Oklahoma', 'Florida'): 962.6326530612245,
 ('Oklahoma', 'Missouri'): 410.9758064516129,
 ('Oklahoma', 'North Carolina'): 900.5869565217391,
 ('Oklahoma', 'Minnesota'): 663.3875,
 ('Oklahoma', 'Arizona'): 869.920245398773,
 ('Oklahoma', 'Utah'): 897.685393258427,
 ('Oklahoma', 'Nevada'): 1026.2797202797203,
 ('Oklahoma', 'Maryland'): 1175.1555555555556,
 ('Oklahoma', 'Michigan'): 894.0,
 ('Oklahoma', 'Pennsylvania'): 1261.0,
 ('Oklahoma', 'Washington'): 1519.0,
 ('Oklahoma', 'Tennessee'): 616.0,
 ('Illinois', 'Utah'): 1251.7435897435898,
 ('Illinois', 'Missouri'): 338.0100565681961,
 ('Illinois', 'Kentucky'): 281.3981937602627,
 ('Illinois', 'Tennesse

## time_diff 컬럼추가

In [67]:
train_df['time_diff']=np.nan

In [68]:
train_df

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,time_diff
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,NaN
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN,NaN
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,NaN
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN,NaN
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN,NaN
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN,NaN
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN,NaN


In [69]:
#train_df에 시간차이의 평균과 거리 평균 컬럼으로 추가
for i in range(len(train_df)):
    if (train_df['Origin_State'].iloc[i], train_df['Destination_State'].iloc[i]) in time_diff_dict.keys():
        train_df['time_diff'].iloc[i] = time_diff_dict[train_df['Origin_State'].iloc[i], train_df['Destination_State'].iloc[i]]

In [70]:
train_df

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,time_diff
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,130.941054
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,261.236467
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN,183.983978
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,834.949679
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,135.909674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN,NaN
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN,NaN
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN,NaN
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN,NaN


## 출발,도착예정시간 결측치 채우기 

In [113]:
for i in range(len(train_df)):
    #출발시간은 데이터가 존재하고, 도착시간은 존재하지않을 때
    if train_df['Estimated_Departure_Time'].iloc[i].astype(str)!='nan' and train_df['Estimated_Arrival_Time'].iloc[i].astype(str)=='nan': 
        train_df['Estimated_Arrival_Time'].iloc[i] = train_df['Estimated_Departure_Time'].iloc[i] + train_df['time_diff'].iloc[i]
     #출발시간은 존재하지 않고, 도착시간은 존재할때    
    elif train_df['Estimated_Departure_Time'].iloc[i].astype(str)=='nan' and train_df['Estimated_Arrival_Time'].iloc[i].astype(str)!='nan': 
        train_df['Estimated_Departure_Time'].iloc[i] = train_df['Estimated_Arrival_Time'].iloc[i] - train_df['time_diff'].iloc[i]


In [115]:
train_df.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time     31786
Estimated_Arrival_Time       31913
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
time_diff                   206854
dtype: int64

In [117]:
train_df[:30]

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,time_diff
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,130.941054
1,TRAIN_000001,8,15,740.0,1024.000000,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,261.236467
2,TRAIN_000002,9,6,1610.0,1805.000000,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN,183.983978
3,TRAIN_000003,7,10,905.0,1735.000000,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,834.949679
4,TRAIN_000004,1,11,900.0,1019.000000,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,135.909674
5,TRAIN_000005,4,13,1545.0,NaN,0,0,EWR,11618,NaN,DCA,11278,Virginia,199.0,Republic Airlines,UA,20452.0,N657RW,Not_Delayed,NaN
6,TRAIN_000006,1,20,1742.0,1903.000000,0,0,EWR,11618,New Jersey,BOS,10721,Massachusetts,200.0,United Air Lines Inc.,UA,NaN,N66825,Not_Delayed,120.928196
7,TRAIN_000007,4,20,1815.0,1955.000000,0,0,ORD,13930,Illinois,MCI,13198,Missouri,403.0,NaN,UA,20304.0,N110SY,NaN,141.636706
8,TRAIN_000008,6,13,1420.0,1550.000000,0,0,BWI,10821,NaN,CLT,11057,North Carolina,361.0,Southwest Airlines Co.,WN,19393.0,N765SW,Not_Delayed,NaN
9,TRAIN_000009,6,6,650.0,838.000000,0,0,LIT,12992,Arkansas,IAH,12266,Texas,374.0,ExpressJet Airlines Inc.,UA,20366.0,N14902,NaN,134.970684
